# CS 335: Introduction to Large Language Models
## Assignment 01
### **Total Marks**: 100
### **Deadline**: Sunday, 3rd March, 2024, 11:59 PM
### **Name**: Write your name here
### **ID**: Write your ID here

#Instructions

1. Please rename your notebook as *Assignment_1_aa1234.ipynb* before the final submission. Notebooks which do not follow appropriate naming convention will not be graded.

2. Please submit your own work. If you have any questions, please feel free to reach out to the course instructors or RA.



# Assignment Overview

In this assignment, you are required to fine tune a LLM model of your that classifies which human value category a textual arguement belongs to. Your model will evaluated against 1-baseline, random-baseline results on the following dataset: test, Nahjalbalagha, Zhihu


# Setup



In [1]:
# IMPORT ALL YOUR LIBRARIES
# SUGGESTED LIBRARIES
import torch
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score

# Download Files


##Evaluator

In [2]:
# DO NOT EDIT
# RUN ONLY ONCE
!wget https://raw.githubusercontent.com/touche-webis-de/touche-code/main/semeval23/human-value-detection/evaluator/evaluator.py

--2024-02-19 19:22:44--  https://raw.githubusercontent.com/touche-webis-de/touche-code/main/semeval23/human-value-detection/evaluator/evaluator.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8708 (8.5K) [text/plain]
Saving to: ‘evaluator.py’

evaluator.py        100%[===================>]   8.50K  --.-KB/s    in 0s      

2024-02-19 19:22:44 (58.0 MB/s) - ‘evaluator.py’ saved [8708/8708]



## 1-Baseline

In [3]:
# DO NOT EDIT
# RUN ONLY ONCE
!wget https://raw.githubusercontent.com/touche-webis-de/touche-code/main/semeval23/human-value-detection/1-baseline/1-baseline.py

--2024-02-19 19:22:44--  https://raw.githubusercontent.com/touche-webis-de/touche-code/main/semeval23/human-value-detection/1-baseline/1-baseline.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3613 (3.5K) [text/plain]
Saving to: ‘1-baseline.py’

1-baseline.py       100%[===================>]   3.53K  --.-KB/s    in 0s      

2024-02-19 19:22:44 (41.0 MB/s) - ‘1-baseline.py’ saved [3613/3613]



## Random-Baseline

In [4]:
# DO NOT EDIT
# RUN ONLY ONCE
!wget https://raw.githubusercontent.com/touche-webis-de/touche-code/main/semeval23/human-value-detection/random-baseline/random-baseline.py

--2024-02-19 19:22:44--  https://raw.githubusercontent.com/touche-webis-de/touche-code/main/semeval23/human-value-detection/random-baseline/random-baseline.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5442 (5.3K) [text/plain]
Saving to: ‘random-baseline.py’

random-baseline.py  100%[===================>]   5.31K  --.-KB/s    in 0s      

2024-02-19 19:22:45 (58.4 MB/s) - ‘random-baseline.py’ saved [5442/5442]



## Dataset Files

In [5]:
# DO NOT EDIT
# RUN ONLY ONCE
!wget https://zenodo.org/api/records/10564870/files-archive

--2024-02-19 19:22:45--  https://zenodo.org/api/records/10564870/files-archive
Resolving zenodo.org (zenodo.org)... 188.184.98.238, 188.185.79.172, 188.184.103.159, ...
Connecting to zenodo.org (zenodo.org)|188.184.98.238|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘files-archive’

files-archive           [               <=>  ]  54.70M  15.6MB/s    in 3.5s    

2024-02-19 19:22:49 (15.6 MB/s) - ‘files-archive’ saved [57353430]



In [6]:
# DO NOT EDIT
# RUN ONLY ONCE
!unzip /content/files-archive -d Dataset

Archive:  /content/files-archive
 extracting: Dataset/value-categories.json  
 extracting: Dataset/arguments-validation-zhihu.tsv  
 extracting: Dataset/level1-labels-training.tsv  
 extracting: Dataset/arguments-test-nahjalbalagha.tsv  
 extracting: Dataset/arguments-test.tsv  
 extracting: Dataset/arguments-training.tsv  
 extracting: Dataset/arguments-validation.tsv  
 extracting: Dataset/labels-test-nahjalbalagha.tsv  
 extracting: Dataset/labels-test-nyt.tsv  
 extracting: Dataset/labels-test.tsv  
 extracting: Dataset/labels-training.tsv  
 extracting: Dataset/level1-labels-test-nahjalbalagha.tsv  
 extracting: Dataset/level1-labels-test.tsv  
 extracting: Dataset/level1-labels-test-nyt.tsv  
 extracting: Dataset/labels-validation.tsv  
 extracting: Dataset/labels-validation-zhihu.tsv  
 extracting: Dataset/meta-arguments-g.tsv  
 extracting: Dataset/meta-arguments-c.tsv  
 extracting: Dataset/meta-arguments-e.tsv  
 extracting: Dataset/meta-arguments-f.tsv  
 extracting: Dataset

In [7]:
!mkdir /content/Dataset/zhihu
!mkdir /content/Dataset/nahjalbalagha
!mkdir /content/Dataset/train
!mkdir /content/Dataset/test
!mkdir /content/Dataset/validation

In [8]:
!mv /content/Dataset/*-zhihu.tsv /content/Dataset/zhihu
!mv /content/Dataset/*-nahjalbalagha.tsv /content/Dataset/nahjalbalagha
!mv /content/Dataset/*-nahjalbalagha.tsv /content/Dataset/nahjalbalagha
!mv /content/Dataset/*-training.tsv /content/Dataset/train
!mv /content/Dataset/*-test.tsv /content/Dataset/test
!mv /content/Dataset/*-validation.tsv /content/Dataset/validation

mv: cannot stat '/content/Dataset/*-nahjalbalagha.tsv': No such file or directory


# Background Information

## Human Value Detection 2023 <br/>
## SemEval 2023 Task 4. ValueEval: Identification of Human Values behind Arguments



Given a textual argument and a human value category, classify whether or not the argument draws on that category. This task uses a set of 20 value categories compiled from the social science literature and described in our [ACL paper](https://webis.de/publications.html#kiesel_2022b). Arguments are given as premise text, conclusion text, and binary stance of the premise to the conclusion ("in favor of" or "against").

The 20 value categories are shown here on Schwartz' value continuum below:

[![JEPBxUu.md.png](https://iili.io/JEPBxUu.md.png)](https://freeimage.host/i/JEPBxUu)







## Data


Data is provided as tab-separated values files with one header line. The arguments-validation.tsv files contain one argument per line: its unique argument ID, the conclusion, the premise's stance towards the conclusion, and the premise itself. Example with tab-separated columns are shown below

<pre><span class="column">Argument ID</span>	<span class="column">Conclusion</span>	<span class="column">Stance</span>	<span class="column">Premise</span>
<span class="column">A01010</span>	<span class="column">We should prohibit school prayer</span>	<span class="column">against</span>	<span class="column">it should be allowed if the student wants to pray as long as it is not interfering with his classes</span>
<span class="column">A01011</span>	<span class="column">We should abolish the three-strikes laws</span>	<span class="column">in favor of</span>	<span class="column">three strike laws can cause young people to be put away for life without a chance to straight out their life</span>
<span class="column">A01012</span>	<span class="column">The use of public defenders should be mandatory</span>	<span class="column">in favor of</span>	<span class="column">the use of public defenders should be mandatory because some people don't have money for a lawyer and this would help those that don't</span>
</pre>

The labels-validation.tsv  files also contain one argument per line: its unique argument ID and one column for each of the 20 value categories with a 1 meaning that the argument resorts to the value category and a 0 that not. Example with tab-separated columns are shown below:

<pre><span class="column">Argument ID</span>	<span class="column">Self-direction: thought</span>	<span class="column">Self-direction: action</span>	<span class="column">Stimulation</span>	<span class="column">Hedonism</span>	<span class="column">Achievement</span>	<span class="column">Power: dominance</span>	<span class="column">Power: resources</span>	<span class="column">Face</span>	<span class="column">Security: personal</span>	<span class="column">Security: societal</span>	<span class="column">Tradition</span>	<span class="column">Conformity: rules</span>	<span class="column">Conformity: interpersonal</span>	<span class="column">Humility</span>	<span class="column">Benevolence: caring</span>	<span class="column">Benevolence: dependability</span>	<span class="column">Universalism: concern</span>	<span class="column">Universalism: nature</span>	<span class="column">Universalism: tolerance</span>	<span class="column">Universalism: objectivity</span>
<span class="column">A01010</span>	<span class="column">1</span>	<span class="column">1</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">1</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">1</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>
<span class="column">A01011</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">1</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">1</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">1</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">1</span>	<span class="column">1</span>
<span class="column">A01012</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">1</span>	<span class="column">0</span>	<span class="column">0</span>	<span class="column">0</span>
</pre>

In addition, there are other datasets for evaluating the robustness of our model: validation-zhihu from the recommendation and hotlist section of the Chinese question-answering website Zhihu, test-nahjalbalagha from and based on the Nahj al-Balagha.



## Evaluation

Runs are evaluated on the basis of F1-score, Precision, and Recall: averaged over all value categories and for each category individually.

## Baseline Results

In [9]:
# DO NOT EDIT
# RUN ONLY ONCE
!mkdir /content/baseline
!mkdir /content/output

### 1-Baseline

#### Test Dataset

In [10]:
# DO NOT EDIT
!python3 /content/1-baseline.py --inputDataset /content/Dataset/test --outputDataset /content/baseline
!python3 evaluator.py --inputDataset /content/Dataset/test/ --inputRun /content/baseline --outputDataset /content/output
!head -n 12 /content/output/evaluation.prototext

Reading /content/Dataset/test/arguments-test.tsv
Labeling 1576 instances
Detected values: {'Power: dominance', 'Humility', 'Universalism: tolerance', 'Self-direction: thought', 'Universalism: objectivity', 'Conformity: interpersonal', 'Face', 'Benevolence: dependability', 'Benevolence: caring', 'Hedonism', 'Security: personal', 'Achievement', 'Self-direction: action', 'Universalism: nature', 'Tradition', 'Universalism: concern', 'Conformity: rules', 'Stimulation', 'Security: societal', 'Power: resources'}
Writing run file
Reading /content/Dataset/test/labels-test.tsv
Reading /content/baseline/run.tsv
Truth labels: 1576
Run labels:   1576
measure {
 key: "F1"
 value: "0.26293020308065357"
}
measure {
 key: "Precision"
 value: "0.15136421319796953"
}
measure {
 key: "Recall"
 value: "1.0"
}


#### Zhihu

In [11]:
# DO NOT EDIT
!python3 /content/1-baseline.py --inputDataset /content/Dataset/zhihu/ --outputDataset /content/baseline
!python3 evaluator.py --inputDataset /content/Dataset/zhihu/ --inputRun /content/baseline --outputDataset /content/output
!head -n 12 /content/output/evaluation.prototext

Reading /content/Dataset/zhihu/arguments-validation-zhihu.tsv
Labeling 100 instances
Detected values: {'Security: personal', 'Benevolence: dependability', 'Universalism: concern', 'Universalism: nature', 'Conformity: interpersonal', 'Conformity: rules', 'Self-direction: thought', 'Face', 'Achievement', 'Security: societal', 'Humility', 'Power: resources', 'Tradition', 'Universalism: objectivity', 'Universalism: tolerance', 'Stimulation', 'Hedonism', 'Benevolence: caring', 'Self-direction: action', 'Power: dominance'}
Writing run file
Reading /content/Dataset/zhihu/labels-validation-zhihu.tsv
Reading /content/baseline/run.tsv
Truth labels: 100
Run labels:   100
measure {
 key: "F1"
 value: "0.2292179045745204"
}
measure {
 key: "Precision"
 value: "0.12944444444444445"
}
measure {
 key: "Recall"
 value: "1.0"
}


#### Nahjalbalagha

In [12]:
# DO NOT EDIT
!python3 /content/1-baseline.py --inputDataset /content/Dataset/nahjalbalagha/ --outputDataset /content/baseline
!python3 evaluator.py --inputDataset /content/Dataset/nahjalbalagha/ --inputRun /content/baseline --outputDataset /content/output
!head -n 12 /content/output/evaluation.prototext

Reading /content/Dataset/nahjalbalagha/arguments-test-nahjalbalagha.tsv
Labeling 279 instances
Detected values: {'Self-direction: action', 'Security: personal', 'Security: societal', 'Tradition', 'Achievement', 'Power: resources', 'Universalism: tolerance', 'Hedonism', 'Universalism: nature', 'Self-direction: thought', 'Conformity: interpersonal', 'Universalism: concern', 'Conformity: rules', 'Universalism: objectivity', 'Humility', 'Power: dominance', 'Stimulation', 'Benevolence: dependability', 'Benevolence: caring', 'Face'}
Writing run file
Reading /content/Dataset/nahjalbalagha/labels-test-nahjalbalagha.tsv
Reading /content/baseline/run.tsv
Truth labels: 279
Run labels:   279
measure {
 key: "F1"
 value: "0.12845882944826426"
}
measure {
 key: "Precision"
 value: "0.0686379928315412"
}
measure {
 key: "Recall"
 value: "1.0"
}


### Random-Baseline


#### Test

In [13]:
# DO NOT EDIT
!python3 /content/random-baseline.py --inputDataset /content/Dataset/test --outputDataset /content/baseline
!python3 evaluator.py --inputDataset /content/Dataset/test/ --inputRun /content/baseline --outputDataset /content/output
!head -n 12 /content/output/evaluation.prototext

Reading /content/Dataset/test/arguments-test.tsv
Labeling 1576 instances
Detected values: {'Benevolence: caring', 'Humility', 'Stimulation', 'Power: resources', 'Security: societal', 'Self-direction: action', 'Self-direction: thought', 'Universalism: nature', 'Face', 'Security: personal', 'Power: dominance', 'Benevolence: dependability', 'Conformity: rules', 'Conformity: interpersonal', 'Universalism: concern', 'Universalism: tolerance', 'Tradition', 'Achievement', 'Universalism: objectivity', 'Hedonism'}
Writing run file
Reading /content/Dataset/test/labels-test.tsv
Reading /content/baseline/run.tsv
Truth labels: 1576
Run labels:   1576
measure {
 key: "F1"
 value: "0.16586590082911043"
}
measure {
 key: "Precision"
 value: "0.15618489179607847"
}
measure {
 key: "Recall"
 value: "0.17682635630588248"
}


#### Zhihu


In [14]:
# DO NOT EDIT
!python3 /content/random-baseline.py --inputDataset /content/Dataset/zhihu/ --outputDataset /content/baseline
!python3 evaluator.py --inputDataset /content/Dataset/zhihu/ --inputRun /content/baseline --outputDataset /content/output
!head -n 12 /content/output/evaluation.prototext

Reading /content/Dataset/zhihu/arguments-validation-zhihu.tsv
Labeling 100 instances
Detected values: {'Face', 'Hedonism', 'Security: societal', 'Tradition', 'Power: resources', 'Stimulation', 'Universalism: concern', 'Self-direction: action', 'Security: personal', 'Conformity: interpersonal', 'Universalism: tolerance', 'Humility', 'Benevolence: dependability', 'Achievement', 'Conformity: rules', 'Universalism: nature', 'Universalism: objectivity', 'Benevolence: caring', 'Self-direction: thought', 'Power: dominance'}
Writing run file
Reading /content/Dataset/zhihu/labels-validation-zhihu.tsv
Reading /content/baseline/run.tsv
Truth labels: 100
Run labels:   100
measure {
 key: "F1"
 value: "0.1498601577315835"
}
measure {
 key: "Precision"
 value: "0.13482683450469005"
}
measure {
 key: "Recall"
 value: "0.16866665372607648"
}


#### Nahjalbalagha

In [15]:
# DO NOT EDIT
!python3 /content/random-baseline.py --inputDataset /content/Dataset/nahjalbalagha/ --outputDataset /content/baseline
!python3 evaluator.py --inputDataset /content/Dataset/nahjalbalagha/ --inputRun /content/baseline --outputDataset /content/output
!head -n 12 /content/output/evaluation.prototext

Reading /content/Dataset/nahjalbalagha/arguments-test-nahjalbalagha.tsv
Labeling 279 instances
Detected values: {'Benevolence: dependability', 'Self-direction: thought', 'Self-direction: action', 'Humility', 'Security: personal', 'Benevolence: caring', 'Conformity: rules', 'Tradition', 'Universalism: nature', 'Security: societal', 'Universalism: tolerance', 'Face', 'Achievement', 'Power: resources', 'Stimulation', 'Universalism: concern', 'Conformity: interpersonal', 'Hedonism', 'Power: dominance', 'Universalism: objectivity'}
Writing run file
Reading /content/Dataset/nahjalbalagha/labels-test-nahjalbalagha.tsv
Reading /content/baseline/run.tsv
Truth labels: 279
Run labels:   279
measure {
 key: "F1"
 value: "0.10457355912331483"
}
measure {
 key: "Precision"
 value: "0.07133237313634852"
}
measure {
 key: "Recall"
 value: "0.19583217793466123"
}


# Tasks

## [20 Points] Task 01 - Load Datasets

In this task, you are required to load the Training, Test, Validation, Nahjalbalagha & Zhihu into seperate dataframes.

In [16]:
# Write your code here

## [10 Points] Task 02 - Define Tokenizer & Model


In this task, you are required to define the Tokenizer and LLM model of your choice.

In [17]:
# Write your code here

## [20 Points] Task 03 - Optimizer & Hyperparameters


In this task, you are required to define the hyperparameters & the optimizer for training your model.

In [18]:
# Write your code here

## [20 Points] Task 04 -  Training Loop


In this task, you are required to implement the training loop for fine tuning your model. You are also required to plot on the same graph: Loss vs Epochs & Accuracy vs Epochs

In [19]:
# Write your code here

## [10 Points]  Task 05 - Model Evaluation: Test Dataset

In this task, you are required your fine tuned model on the Test dataset using ``evaluator.py`` and compare your results with random and 1-baseline.

In [20]:
#Write your code

## [10 Points]  Task 06 - Model Evaluation: Zhihu Dataset

In this task, you are required your fine tuned model on the Zhihu
 dataset using ``evaluator.py`` and compare your results with random and 1-baseline.

## [10 Points]  Task 07 - Model Evaluation: Nahjalbalagha Dataset

In this task, you are required your fine tuned model on the Nahjalbalagha dataset using ``evaluator.py`` and compare your results with random and 1-baseline.

# References

In this section, cite any resources or references that you use for solving this assignment.
